In [1]:
!pip install flask-ngrok
!pip install pyngrok
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 26.4 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import google.generativeai as genai
import re
from IPython.display import display, Markdown
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Initialize the Flask app
app = Flask(__name__)

# Set up ngrok for public URL access
ngrok.set_auth_token("YOUR_NGROK_API_KEY")
public_url = ngrok.connect(5000).public_url

# Configure the Gemini API
genai.configure(api_key="YOUR_GEMINI_API_KEY")


# Selenium function to fetch dynamic location URLs
def get_location_selenium(place_name, city_name):
    query = f"{place_name} {city_name}"
    url = f"https://www.google.com/maps/search/{query.replace(' ', '+')}"

    # Set up Chrome options
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless")  # Run in headless mode if you don't need a visible browser
    options.add_argument("--user-data-dir=/tmp/chrome_user_data")  # Specify a unique user data directory

    # Path to your ChromeDriver
    driver = webdriver.Chrome(service=Service(), options=options)

    try:
        # Open the URL
        driver.get(url)
        driver.implicitly_wait(10)  # Wait for the page to load

        # Get the current URL of the Google Maps result
        location_url = driver.current_url
        driver.quit()
        return location_url
    except Exception as e:
        print(f"Error fetching location: {e}")
        driver.quit()
        return "Error fetching location"



@app.route("/gemini", methods=["POST"])
def gemini_route():
    data = request.get_json()
    country, city = data.get("country"), data.get("city")
    category, category_type = data.get("category"), data.get("category_type")

    if not all([country, city, category, category_type]):
        return jsonify({"error": "Missing required fields"}), 400

    category_type_str = ", ".join(category_type)

    prompt = (
        f"Provide a list of top 10 most popular and well-known places in {city}, {country} that match the following details:\n"
        f"Category: {category}\n"
        f"Types: {category_type_str}\n"
        f"Return information about each place, including name, rating, average price, working time, "
        f"description, and location on the map url."

        f"Example of the response should be in the same format :\n"
        f"""**1. Spazio**

          * **Name:** Spazio
          * **Rating:** 4.5/5
          * **Average Price:** SAR 200-500
          * **Working Time:** 12 PM - 12 AM
          * **Description:** A modern Italian restaurant with a focus on authentic flavors and high-quality ingredients.
          * **Location:** https://goo.gl/maps/tHHU2u5zggejy2y97
        """
    )

    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        print(response.text)

        # Process Gemini API response
        places = []
        if response and response.text:
            entries = re.split(r"\n\n(?=\*\*\d+\.\s)", response.text)  # Split by entries
            for entry in entries:
                place = {}

                # Extract place name
                name_match = re.search(r"\*\*Name:\*\* ([^\n]+)", entry)
                place["place_name"] = name_match.group(1) if name_match else "Unnamed Place"
                place_name = place["place_name"]

                # Fetch dynamic location URL using Selenium
                place["location"] = get_location_selenium(place_name, city)

                # Extract rating
                rating_match = re.search(r"\*\*Rating:\*\* ([^\n]+)", entry)
                place["rating"] = rating_match.group(1) if rating_match else "Not available"

                # Extract average price
                price_match = re.search(r"\*\*Average Price:\*\* ([^\n]+)", entry)
                place["average_price"] = price_match.group(1) if price_match else "Not available"

                # Extract working time
                working_time_match = re.search(r"\*\*Working Time:\*\* ([^\n]+)", entry)
                place["working_time"] = working_time_match.group(1) if working_time_match else "Not available"

                # Extract description
                description_match = re.search(r"\*\*Description:\*\* ([^\n]+)", entry)
                place["description"] = description_match.group(1) if description_match else "Not available"

                # Add to places list with ordered fields
                ordered_place = {
                    "place_name": place["place_name"],
                    "average_price": place["average_price"],
                    "description": place["description"],
                    "rating": place["rating"],
                    "working_time": place["working_time"],
                    "location": place["location"],

                }
                places.append(ordered_place)

                 # Log to Colab notebook
            display(Markdown(f"### Request:\n- Country: {country}\n- City: {city}\n- Category: {category}\n- Category Type: {category_type}"))
            display(Markdown(f"### Response:\n{jsonify({'places': places}).get_data(as_text=True)}"))

            return jsonify({"places": places}), 200
        else:
            return jsonify({"error": "No response from Gemini API"}), 500
    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": "Failed to process request"}), 500

@app.route('/gemini_search', methods=['POST'])
def gemini_search_route():
    data = request.get_json()
    country, city, description = data.get("country"), data.get("city"), data.get("description")

    if not all([country, city, description]):
        return jsonify({"error": "Missing required fields"}), 400


    prompt = (
        f"Provide a list of top 10 most popular and highly recommended places in {city}, {country} that match the following details:\n"
        f"description: {description}\n"
        f"Return information about each place, including name, rating, average price, working time, "
        f"description, and location on the map url."

        f"Example of the response should be in same format :\n"
        f"""**1. Spazio**


          * **Name:** Spazio
          * **Rating:** 4.5/5
          * **Average Price:** SAR 200-500
          * **Working Time:** 12 PM - 12 AM
          * **Description:** A modern Italian restaurant with a focus on authentic flavors and high-quality ingredients.
          * **Location:** https://goo.gl/maps/tHHU2u5zggejy2y97

          **2. Cipriani**


          * **Name:** Cipriani
          * **Rating:** 4.8/5
          * **Average Price:** SAR 300-700
          * **Working Time:** 12 PM - 12 AM
          * **Description:** An upscale Italian restaurant known for its sophisticated ambiance and exceptional cuisine.
          * **Location:** https://goo.gl/maps/27Bi7tKMKz1ySmBEA"""
    )


    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        print(response.text)

        # Process Gemini API response
        places = []
        if response and response.text:
            entries = re.split(r"\n\n(?=\*\*\d+\.\s)", response.text)  # Split by entries
            for entry in entries:
                place = {}

                # Extract place name
                name_match = re.search(r"\*\*Name:\*\* ([^\n]+)", entry)
                place["place_name"] = name_match.group(1) if name_match else "Unnamed Place"
                place_name = place["place_name"]

                # Fetch dynamic location URL using Selenium
                place["location"] = get_location_selenium(place_name, city)

                # Extract rating
                rating_match = re.search(r"\*\*Rating:\*\* ([^\n]+)", entry)
                place["rating"] = rating_match.group(1) if rating_match else "Not available"

                # Extract average price
                price_match = re.search(r"\*\*Average Price:\*\* ([^\n]+)", entry)
                place["average_price"] = price_match.group(1) if price_match else "Not available"

                # Extract working time
                working_time_match = re.search(r"\*\*Working Time:\*\* ([^\n]+)", entry)
                place["working_time"] = working_time_match.group(1) if working_time_match else "Not available"

                # Extract description
                description_match = re.search(r"\*\*Description:\*\* ([^\n]+)", entry)
                place["description"] = description_match.group(1) if description_match else "Not available"

                # Add to places list with ordered fields
                ordered_place = {
                    "place_name": place["place_name"],
                    "average_price": place["average_price"],
                    "description": place["description"],
                    "rating": place["rating"],
                    "working_time": place["working_time"],
                    "location": place["location"],

                }
                places.append(ordered_place)

            # Log to Colab notebook
            display(Markdown(f"### Request:\n- Country: {country}\n- City: {city}\n- Description: {description}"))
            display(Markdown(f"### Response:\n{jsonify({'places': places}).get_data(as_text=True)}"))

            return jsonify({"places": places}), 200
        else:
            return jsonify({"error": "No response from Gemini API"}), 500
    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": "Failed to process request"}), 500

if __name__ == "__main__":
    print(f"App running at {public_url}")
    app.run(port=5000)


App running at https://83fc-35-192-44-145.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
